In [1]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark=SparkSession.builder.getOrCreate()
!wget https://data.cityofchicago.org/api/views/ijzp-q8t2/rows.csv?accessType=DOWNLOAD
!mv rows.csv\?accessType\=DOWNLOAD reported_crimes.csv
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:12 http://security.ubuntu.com/ubu

In [3]:
spark

In [0]:
df=spark.read.csv('multilabel_train.csv',header=True)

In [57]:
df.show(5)

+--------------------+--------------------+-----+------------+-------+------+------+-------------+
|                  id|        comment_text|toxic|severe_toxic|obscene|threat|insult|identity_hate|
+--------------------+--------------------+-----+------------+-------+------+------+-------------+
|    0000997932d777bf|         Explanation| null|        null|   null|  null|  null|         null|
|Why the edits mad...| just closure on ...|    0|           0|      0|     0|     0|            0|
|    000103f0d9cfb60f|D'aww! He matches...|    0|           0|      0|     0|     0|            0|
|    000113f07ec002fd|Hey man, I'm real...|    0|           0|      0|     0|     0|            0|
|    0001b41b1c6bb37e|                   "| null|        null|   null|  null|  null|         null|
+--------------------+--------------------+-----+------------+-------+------+------+-------------+
only showing top 5 rows



In [58]:
import pyspark.sql.functions as F
df=df.withColumn('idlength',F.length(F.col('id')))
df=df.na.drop()
df.show(5)

+--------------------+--------------------+-----+------------+-------+------+------+-------------+--------+
|                  id|        comment_text|toxic|severe_toxic|obscene|threat|insult|identity_hate|idlength|
+--------------------+--------------------+-----+------------+-------+------+------+-------------+--------+
|Why the edits mad...| just closure on ...|    0|           0|      0|     0|     0|            0|      98|
|    000103f0d9cfb60f|D'aww! He matches...|    0|           0|      0|     0|     0|            0|      16|
|    000113f07ec002fd|Hey man, I'm real...|    0|           0|      0|     0|     0|            0|      16|
|    0001d958c54c6e35|You, sir, are my ...|    0|           0|      0|     0|     0|            0|      16|
|Congratulations f...| use the tools we...|    0|           0|      0|     0|     0|            0|      31|
+--------------------+--------------------+-----+------------+-------+------+------+-------------+--------+
only showing top 5 rows



In [0]:
df=df.where(F.col('idlength')==16)

In [60]:
df.show(10)

+----------------+--------------------+-----+------------+-------+------+------+-------------+--------+
|              id|        comment_text|toxic|severe_toxic|obscene|threat|insult|identity_hate|idlength|
+----------------+--------------------+-----+------------+-------+------+------+-------------+--------+
|000103f0d9cfb60f|D'aww! He matches...|    0|           0|      0|     0|     0|            0|      16|
|000113f07ec002fd|Hey man, I'm real...|    0|           0|      0|     0|     0|            0|      16|
|0001d958c54c6e35|You, sir, are my ...|    0|           0|      0|     0|     0|            0|      16|
|0002bcb3da6cb337|COCKSUCKER BEFORE...|    1|           1|      1|     0|     1|            0|      16|
|00031b1e95af7921|Your vandalism to...|    0|           0|      0|     0|     0|            0|      16|
|00037261f536c51d|Sorry if the word...|    0|           0|      0|     0|     0|            0|      16|
|00040093b2687caa|alignment on this...|    0|           0|      

In [0]:
df=df.drop('words')

In [62]:
df.show(10)

+----------------+--------------------+-----+------------+-------+------+------+-------------+--------+
|              id|        comment_text|toxic|severe_toxic|obscene|threat|insult|identity_hate|idlength|
+----------------+--------------------+-----+------------+-------+------+------+-------------+--------+
|000103f0d9cfb60f|D'aww! He matches...|    0|           0|      0|     0|     0|            0|      16|
|000113f07ec002fd|Hey man, I'm real...|    0|           0|      0|     0|     0|            0|      16|
|0001d958c54c6e35|You, sir, are my ...|    0|           0|      0|     0|     0|            0|      16|
|0002bcb3da6cb337|COCKSUCKER BEFORE...|    1|           1|      1|     0|     1|            0|      16|
|00031b1e95af7921|Your vandalism to...|    0|           0|      0|     0|     0|            0|      16|
|00037261f536c51d|Sorry if the word...|    0|           0|      0|     0|     0|            0|      16|
|00040093b2687caa|alignment on this...|    0|           0|      

In [0]:
df=df.drop('idlength')

In [0]:
df=df.withColumn("lowered_text",F.lower(F.col("comment_text")))

In [65]:
df.show(5)

+----------------+--------------------+-----+------------+-------+------+------+-------------+--------------------+
|              id|        comment_text|toxic|severe_toxic|obscene|threat|insult|identity_hate|        lowered_text|
+----------------+--------------------+-----+------------+-------+------+------+-------------+--------------------+
|000103f0d9cfb60f|D'aww! He matches...|    0|           0|      0|     0|     0|            0|d'aww! he matches...|
|000113f07ec002fd|Hey man, I'm real...|    0|           0|      0|     0|     0|            0|hey man, i'm real...|
|0001d958c54c6e35|You, sir, are my ...|    0|           0|      0|     0|     0|            0|you, sir, are my ...|
|0002bcb3da6cb337|COCKSUCKER BEFORE...|    1|           1|      1|     0|     1|            0|cocksucker before...|
|00031b1e95af7921|Your vandalism to...|    0|           0|      0|     0|     0|            0|your vandalism to...|
+----------------+--------------------+-----+------------+-------+------

In [0]:
df=df.cache()

In [16]:
df.show(5)

+----------------+--------------------+-----+------------+-------+------+------+-------------+--------------------+
|              id|        comment_text|toxic|severe_toxic|obscene|threat|insult|identity_hate|        lowered_text|
+----------------+--------------------+-----+------------+-------+------+------+-------------+--------------------+
|000103f0d9cfb60f|D'aww! He matches...|    0|           0|      0|     0|     0|            0|d'aww! he matches...|
|000113f07ec002fd|Hey man, I'm real...|    0|           0|      0|     0|     0|            0|hey man, i'm real...|
|0001d958c54c6e35|You, sir, are my ...|    0|           0|      0|     0|     0|            0|you, sir, are my ...|
|0002bcb3da6cb337|COCKSUCKER BEFORE...|    1|           1|      1|     0|     1|            0|cocksucker before...|
|00031b1e95af7921|Your vandalism to...|    0|           0|      0|     0|     0|            0|your vandalism to...|
+----------------+--------------------+-----+------------+-------+------

In [0]:
from pyspark.ml.feature import RegexTokenizer
tokenizer=RegexTokenizer(inputCol='lowered_text',outputCol="tokens",pattern="\W+")

In [0]:
df=tokenizer.transform(df)

In [69]:
df.show(10)

+----------------+--------------------+-----+------------+-------+------+------+-------------+--------------------+--------------------+
|              id|        comment_text|toxic|severe_toxic|obscene|threat|insult|identity_hate|        lowered_text|              tokens|
+----------------+--------------------+-----+------------+-------+------+------+-------------+--------------------+--------------------+
|000103f0d9cfb60f|D'aww! He matches...|    0|           0|      0|     0|     0|            0|d'aww! he matches...|[d, aww, he, matc...|
|000113f07ec002fd|Hey man, I'm real...|    0|           0|      0|     0|     0|            0|hey man, i'm real...|[hey, man, i, m, ...|
|0001d958c54c6e35|You, sir, are my ...|    0|           0|      0|     0|     0|            0|you, sir, are my ...|[you, sir, are, m...|
|0002bcb3da6cb337|COCKSUCKER BEFORE...|    1|           1|      1|     0|     1|            0|cocksucker before...|[cocksucker, befo...|
|00031b1e95af7921|Your vandalism to...|  

In [70]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- comment_text: string (nullable = true)
 |-- toxic: string (nullable = true)
 |-- severe_toxic: string (nullable = true)
 |-- obscene: string (nullable = true)
 |-- threat: string (nullable = true)
 |-- insult: string (nullable = true)
 |-- identity_hate: string (nullable = true)
 |-- lowered_text: string (nullable = true)
 |-- tokens: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [0]:
from pyspark.ml.feature import CountVectorizer
vectorizer=CountVectorizer(inputCol="tokens",outputCol="countvectorized")

In [0]:
df=df.na.drop()

In [74]:
df.show(2)

+----------------+--------------------+-----+------------+-------+------+------+-------------+--------------------+--------------------+
|              id|        comment_text|toxic|severe_toxic|obscene|threat|insult|identity_hate|        lowered_text|              tokens|
+----------------+--------------------+-----+------------+-------+------+------+-------------+--------------------+--------------------+
|000103f0d9cfb60f|D'aww! He matches...|    0|           0|      0|     0|     0|            0|d'aww! he matches...|[d, aww, he, matc...|
|000113f07ec002fd|Hey man, I'm real...|    0|           0|      0|     0|     0|            0|hey man, i'm real...|[hey, man, i, m, ...|
+----------------+--------------------+-----+------------+-------+------+------+-------------+--------------------+--------------------+
only showing top 2 rows



In [0]:
df=vectorizer.fit(df).transform(df)

In [76]:
df.show(2)

+----------------+--------------------+-----+------------+-------+------+------+-------------+--------------------+--------------------+--------------------+
|              id|        comment_text|toxic|severe_toxic|obscene|threat|insult|identity_hate|        lowered_text|              tokens|     countvectorized|
+----------------+--------------------+-----+------------+-------+------+------+-------------+--------------------+--------------------+--------------------+
|000103f0d9cfb60f|D'aww! He matches...|    0|           0|      0|     0|     0|            0|d'aww! he matches...|[d, aww, he, matc...|(69147,[1,12,22,4...|
|000113f07ec002fd|Hey man, I'm real...|    0|           0|      0|     0|     0|            0|hey man, i'm real...|[hey, man, i, m, ...|(69147,[0,1,2,5,6...|
+----------------+--------------------+-----+------------+-------+------+------+-------------+--------------------+--------------------+--------------------+
only showing top 2 rows



In [0]:
df=df.withColumn('con',F.concat(F.col('toxic'),F.col('severe_toxic'),F.col('obscene'),F.col('threat'),F.col('insult'),F.col('identity_hate')))

In [84]:
df.show(5)

+----------------+--------------------+-----+------------+-------+------+------+-------------+--------------------+--------------------+--------------------+------+
|              id|        comment_text|toxic|severe_toxic|obscene|threat|insult|identity_hate|        lowered_text|              tokens|     countvectorized|   con|
+----------------+--------------------+-----+------------+-------+------+------+-------------+--------------------+--------------------+--------------------+------+
|000103f0d9cfb60f|D'aww! He matches...|    0|           0|      0|     0|     0|            0|d'aww! he matches...|[d, aww, he, matc...|(69147,[1,12,22,4...|000000|
|000113f07ec002fd|Hey man, I'm real...|    0|           0|      0|     0|     0|            0|hey man, i'm real...|[hey, man, i, m, ...|(69147,[0,1,2,5,6...|000000|
|0001d958c54c6e35|You, sir, are my ...|    0|           0|      0|     0|     0|            0|you, sir, are my ...|[you, sir, are, m...|(69147,[3,9,15,16...|000000|
|0002bcb3d

In [0]:
df=df.where(F.length('id')==16)

In [103]:
df.show()

+----------------+--------------------+-----+------------+-------+------+------+-------------+--------------------+--------------------+--------------------+------+
|              id|        comment_text|toxic|severe_toxic|obscene|threat|insult|identity_hate|        lowered_text|              tokens|     countvectorized|   con|
+----------------+--------------------+-----+------------+-------+------+------+-------------+--------------------+--------------------+--------------------+------+
|000103f0d9cfb60f|D'aww! He matches...|    0|           0|      0|     0|     0|            0|d'aww! he matches...|[d, aww, he, matc...|(69147,[1,12,22,4...|000000|
|000113f07ec002fd|Hey man, I'm real...|    0|           0|      0|     0|     0|            0|hey man, i'm real...|[hey, man, i, m, ...|(69147,[0,1,2,5,6...|000000|
|0001d958c54c6e35|You, sir, are my ...|    0|           0|      0|     0|     0|            0|you, sir, are my ...|[you, sir, are, m...|(69147,[3,9,15,16...|000000|
|0002bcb3d

In [0]:
df=df.where(F.length('comment_text')>5)

In [0]:
df=df.filter(~df.id.contains(' '))

In [0]:
df=df.where(F.length('con')==6)

In [0]:
not_okay=df.where(F.col('con')!='000000')

In [0]:
okay=df.where(F.col('con')=='000000')

In [119]:
not_okay.count()

7518

In [120]:
okay.count()

56518

In [0]:
df=not_okay.union(okay.limit(7518))

In [125]:
df.show(5)

+----------------+--------------------+-----+------------+-------+------+------+-------------+--------------------+--------------------+--------------------+------+
|              id|        comment_text|toxic|severe_toxic|obscene|threat|insult|identity_hate|        lowered_text|              tokens|     countvectorized|   con|
+----------------+--------------------+-----+------------+-------+------+------+-------------+--------------------+--------------------+--------------------+------+
|0002bcb3da6cb337|COCKSUCKER BEFORE...|    1|           1|      1|     0|     1|            0|cocksucker before...|[cocksucker, befo...|(69147,[3,15,25,1...|111010|
|00190820581d90ce|FUCK YOUR FILTHY ...|    1|           0|      1|     0|     1|            0|fuck your filthy ...|[fuck, your, filt...|(69147,[0,10,21,6...|101010|
|001dc38a83d420cf|GET FUCKED UP. GE...|    1|           0|      1|     0|     0|            0|get fucked up. ge...|[get, fucked, up,...|(69147,[1,3,4,9,6...|101000|
|0020e7119

In [126]:
df.groupBy('con').count().show()

+------+-----+
|   con|count|
+------+-----+
|100010|  616|
|110000|   15|
|000000| 7518|
|101100|    6|
|110100|    3|
|110011|    5|
|001000|  111|
|111110|   22|
|111000|   82|
|100110|    4|
|100000| 2522|
|100101|    4|
|111001|    2|
|110001|    1|
|000011|   11|
|000100|   10|
|101010| 1777|
|100011|   64|
|"00000|    1|
|001010|   72|
+------+-----+
only showing top 20 rows



In [0]:
df=df.where(~F.col('con').contains('\"'))

In [130]:
df.groupBy('con').count().show()

+------+-----+
|   con|count|
+------+-----+
|100010|  616|
|110000|   15|
|000000| 7518|
|101100|    6|
|110100|    3|
|110011|    5|
|001000|  111|
|111110|   22|
|111000|   82|
|100110|    4|
|100000| 2522|
|100101|    4|
|111001|    2|
|110001|    1|
|000011|   11|
|000100|   10|
|101010| 1777|
|100011|   64|
|001010|   72|
|110010|    6|
+------+-----+
only showing top 20 rows



In [0]:
from pyspark.ml.feature import StringIndexer

In [0]:
indexer=StringIndexer(inputCol='con',outputCol='label')

In [0]:
indexer=indexer.fit(df)

In [0]:
df=indexer.transform(df)

In [141]:
df.count()

15035

In [0]:
from pyspark.ml.classification import NaiveBayes
model=NaiveBayes(featuresCol='countvectorized',labelCol='label')

In [0]:
train,test=df.randomSplit([0.8,0.2],666)

In [146]:
train.count()

11963

In [147]:
test.count()

3072

In [0]:
model=model.fit(train)

In [152]:
model.transform(test).show()

+----------------+--------------------+-----+------------+-------+------+------+-------------+--------------------+--------------------+--------------------+------+-----+--------------------+--------------------+----------+
|              id|        comment_text|toxic|severe_toxic|obscene|threat|insult|identity_hate|        lowered_text|              tokens|     countvectorized|   con|label|       rawPrediction|         probability|prediction|
+----------------+--------------------+-----+------------+-------+------+------+-------------+--------------------+--------------------+--------------------+------+-----+--------------------+--------------------+----------+
|0028d62e8a5629aa|All of my edits a...|    1|           0|      1|     0|     1|            0|all of my edits a...|[all, of, my, edi...|(69147,[0,2,3,4,5...|101010|  2.0|[-428.60569620718...|[0.99981675278207...|       0.0|
|00cce3996a16c116|Keep your eye on ...|    1|           0|      0|     0|     1|            0|keep your 

In [0]:
train_pred=model.transform(train)

In [156]:
train_pred.groupBy('label','prediction').count().where(F.col('label')==F.col('prediction')).show

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|  634|
| 10.0|      10.0|    8|
| 27.0|      27.0|    1|
|  2.0|       2.0|  742|
|  6.0|       6.0|    4|
| 25.0|      25.0|    1|
|  4.0|       4.0|   17|
| 20.0|      20.0|    2|
|  8.0|       8.0|    7|
| 13.0|      13.0|    1|
|  5.0|       5.0|  150|
| 33.0|      33.0|    1|
|  0.0|       0.0| 5949|
| 30.0|      30.0|    1|
|  3.0|       3.0|   27|
+-----+----------+-----+



In [158]:
train_pred.groupBy('label','prediction').count().where(F.col('label')!=F.col('prediction')).show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
| 19.0|       5.0|    1|
| 10.0|       1.0|   10|
|  2.0|       0.0|  414|
|  6.0|       1.0|   88|
|  3.0|       5.0|   23|
| 23.0|       0.0|    6|
|  8.0|       5.0|   19|
| 10.0|       2.0|   35|
| 20.0|       5.0|    1|
| 14.0|       0.0|   23|
| 19.0|       0.0|    4|
| 11.0|       1.0|   27|
| 23.0|       1.0|    1|
| 22.0|       1.0|    4|
| 12.0|       4.0|    1|
| 32.0|       0.0|    1|
|  5.0|       1.0|   59|
| 10.0|       0.0|    8|
|  5.0|       2.0|  149|
| 21.0|       0.0|    2|
+-----+----------+-----+
only showing top 20 rows



In [0]:
test_pred=model.transform(test)

In [161]:
test_pred.groupBy('label','prediction').count().where(F.col('label')==F.col('prediction')).show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|   92|
|  2.0|       2.0|  123|
|  4.0|       4.0|    1|
|  5.0|       5.0|   22|
|  0.0|       0.0| 1504|
|  3.0|       3.0|    2|
+-----+----------+-----+



In [162]:
test_pred.groupBy('label','prediction').count().where(F.col('label')!=F.col('prediction')).show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
| 10.0|       1.0|    1|
|  2.0|       0.0|  128|
|  6.0|       1.0|   22|
|  3.0|       5.0|    1|
| 23.0|       0.0|    3|
|  8.0|       5.0|    7|
|  5.0|       8.0|    1|
| 10.0|       2.0|    4|
| 14.0|       0.0|   12|
| 19.0|       0.0|    4|
| 11.0|       1.0|    8|
| 14.0|       2.0|    1|
|  5.0|       1.0|   12|
| 10.0|       0.0|    1|
|  5.0|       2.0|   56|
| 13.0|       0.0|    7|
| 16.0|       2.0|    3|
|  9.0|       0.0|   20|
| 26.0|       1.0|    1|
|  7.0|       1.0|    5|
+-----+----------+-----+
only showing top 20 rows

